<a href="https://colab.research.google.com/github/kinjaljoshi/aiagent_data_exploration/blob/main/faiss_store_metadata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install faiss-cpu sentence-transformers google-cloud-bigquery langchain_community

In [16]:
import nltk
nltk.download('punkt', force=True)
nltk.download('punkt_tab', force=True)
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [5]:
from google.colab import auth
auth.authenticate_user()

PROJECT_ID = 'llm-text-to-sql-445914'
!gcloud config set project $PROJECT_ID

Updated property [core/project].


## Fetch Metadata from BigQuery

In [8]:
from google.cloud import bigquery
import json
from collections import defaultdict

client = bigquery.Client(project="llm-text-to-sql-445914")

# Define Project & Dataset
project_id = "llm-text-to-sql-445914"
dataset_id = "llm_text_to_sql"

# Fetch all table names in the dataset
query_tables = f"""
    SELECT table_name
    FROM `{project_id}.{dataset_id}.INFORMATION_SCHEMA.TABLES`
"""

tables_result = client.query(query_tables).result()
table_names = [row.table_name for row in tables_result]

# Initialize Dictionary for Table Definitions
table_definitions = {"tables": []}

# Iterate over tables to fetch metadata
for table_name in table_names:
    query = f"""
        SELECT
            table_name,
            column_name,
            IFNULL(description, 'No Description') AS column_description
        FROM
            `{project_id}.{dataset_id}.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS`
        WHERE
            table_name = '{table_name}'
    """

    query_results = client.query(query).result()

    # Group columns into table structure
    table_dict = {
        "table_name": table_name,
        "table_description": f"Metadata for table {table_name}.",  # You can customize this if descriptions exist
        "columns": []
    }

    for row in query_results:
        table_dict["columns"].append({
            "column_name": row.column_name,
            "column_description": row.column_description
        })

    # Append formatted table data
    table_definitions["tables"].append(table_dict)

# Print the final JSON structure
print(json.dumps(table_definitions, indent=4))


{
    "tables": [
        {
            "table_name": "XX_COMPANY_ITEMS",
            "table_description": "Metadata for table XX_COMPANY_ITEMS.",
            "columns": [
                {
                    "column_name": "item_id",
                    "column_description": "Unique item identifier"
                },
                {
                    "column_name": "description",
                    "column_description": "Item Description"
                },
                {
                    "column_name": "uom",
                    "column_description": "Unit Of Measure"
                },
                {
                    "column_name": "length",
                    "column_description": "Length dimension of item"
                },
                {
                    "column_name": "width",
                    "column_description": "width dimension of item"
                },
                {
                    "column_name": "height",
                    "column_

## Store Metadata in FAISS using langchain Documents

In [ ]:
import json
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings

# Load Sentence Transformer Model
embedding_model = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")


# Step 1: Convert Each Table into a Single LangChain Document
documents = []

for table in table_definitions["tables"]:
    # Format table description
    table_text = f"Table: {table['table_name']}\nDescription: {table['table_description']}\n\nColumns:\n"

    # Add all columns into the same document
    for column in table["columns"]:
        table_text += f"  - {column['column_name']}: {column['column_description']}\n"

    # Store full table structure in one Document
    doc = Document(
        page_content=table_text,
        metadata={"table_name": table["table_name"]}
    )
    documents.append(doc)

# Step 2: Store in FAISS Using LangChain's FAISS Wrapper
vector_db = FAISS.from_documents(documents, embedding_model)

# Save FAISS Index
vector_db.save_local("faiss_table_index")

print(f"Stored {len(documents)} table definitions in FAISS as single documents.")


## Cleanup search text and get key words

In [18]:
# Define custom stop words (action verbs & common words)
custom_stopwords = set(stopwords.words("english")).union({"get", "put", "fetch", "retrieve", "show", "find"})

# Input Query
query_text = "Get Warehouse and item and inventory in each warehouse"

# Tokenize & Convert to Lowercase
words = word_tokenize(query_text.lower())

# Remove Stop Words & Duplicates (Preserve Order)
filtered_words = []
for word in words:
    if word not in custom_stopwords and word not in filtered_words:
        filtered_words.append(word)

# Display Extracted Key Terms
print("Key Terms:", filtered_words)


Key Terms: ['warehouse', 'item', 'inventory']


## Retrieve Table data from FAISS using langchain vector_db similarity search

In [20]:
# Load FAISS Index
vector_db = FAISS.load_local("faiss_table_index", embedding_model, allow_dangerous_deserialization=True)


# Perform FAISS Search
final_docs = []
for search_text in filtered_words:
  retrieved_docs = vector_db.similarity_search(search_text, k=1)  # Retrieve top-1 match
  final_docs.append(retrieved_docs[0])
# Extract Matched Document
#matched_document = retrieved_docs[0]

# Display Full Table Definition
print("\n **Search Result:**")
for doc in final_docs:
  print(doc.page_content)



 **Search Result:**
Table: XX_COMPANY_WAREHOUSES
Description: Metadata for table XX_COMPANY_WAREHOUSES.

Columns:
  - warehouse_id: Unique Warehouse identifier
  - warehouse_name: Warehouse Name
  - location: Warehouse Location

Table: XX_COMPANY_ITEMS
Description: Metadata for table XX_COMPANY_ITEMS.

Columns:
  - item_id: Unique item identifier
  - description: Item Description
  - uom: Unit Of Measure
  - length: Length dimension of item
  - width: width dimension of item
  - height: height dimension of item
  - weight: weight of item

Table: XX_COMPANY_INVENTORY
Description: Metadata for table XX_COMPANY_INVENTORY.

Columns:
  - inventory_id: Unique inventory record id
  - warehouse_id: warehouse identifier
  - item_id: item identifier
  - stock_quantity: On hand stock quantity
  - last_updated: Last Updated timestamp

